the session will be uploaded from the fetch_data

In [3]:
import sys
sys.path.append('../scripts')   # allows pysthon to find scripts folder

from fetch_data import load_session 

session = load_session(2025, "Monza", "R")
laps = session.laps.copy()

print(laps.head())
print(laps.columns)
print(laps[['Driver', 'LapNumber', 'LapTime', 'Compound', 'TyreLife']].head(10))


core           INFO 	Loading data for Italian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '81', '16', '63', '44', '23', '5', '12', '6', '55', '87', '22', '30', '31', '10', '43', '18', '14', '27']


                    Time Driver DriverNumber                LapTime  \
0 0 days 00:59:17.271000    VER            1 0 days 00:01:27.159000   
1 0 days 01:00:42.130000    VER            1 0 days 00:01:24.859000   
2 0 days 01:02:05.642000    VER            1 0 days 00:01:23.512000   
3 0 days 01:03:28.904000    VER            1 0 days 00:01:23.262000   
4 0 days 01:04:52.492000    VER            1 0 days 00:01:23.588000   

   LapNumber  Stint PitOutTime PitInTime            Sector1Time  \
0        1.0    1.0        NaT       NaT                    NaT   
1        2.0    1.0        NaT       NaT 0 days 00:00:28.457000   
2        3.0    1.0        NaT       NaT 0 days 00:00:27.212000   
3        4.0    1.0        NaT       NaT 0 days 00:00:27.375000   
4        5.0    1.0        NaT       NaT 0 days 00:00:27.520000   

             Sector2Time  ... FreshTyre             Team  \
0 0 days 00:00:29.088000  ...      True  Red Bull Racing   
1 0 days 00:00:28.843000  ...      True  Red Bull 